In [ ]:
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.metrics import  jaccard_similarity_score, jaccard_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ratings = pd.read_csv("/content/drive/My Drive/2-1/88956262RecommenderSystems/ml-latest-small/ratings.csv")
movies = pd.read_csv("/content/drive/My Drive/2-1/88956262RecommenderSystems/ml-latest-small/movies.csv")
links = pd.read_csv("/content/drive/My Drive/2-1/88956262RecommenderSystems/ml-latest-small/links.csv")
tags = pd.read_csv("/content/drive/My Drive/2-1/88956262RecommenderSystems/ml-latest-small/tags.csv")
genres = ["Action","Adventure","Animation","Children","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western","(no genres listed)"]
genres1 = ["title","Action","Adventure","Animation","Children","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western","(no genres listed)"]
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
rating_list = [x for x in np.arange(0.5,5+0.5,0.5)]
rating_list_ = rating_list.copy()
rating_list_.insert(0,'title')
print(rating_list)
print(rating_list_)

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
['title', 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [ ]:
rating_2= ratings.copy()
df1 = rating_2.groupby('movieId')['rating'].apply(list).reset_index(name='rating_list')
df1

,movieId,rating_list
0,1,"[4.0, 4.0, 4.5, 2.5, 4.5, 3.5, 4.0, 3.5, 3.0, ..."
1,2,"[4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.5, 4.0, ..."
2,3,"[4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, ..."
3,4,"[3.0, 3.0, 3.0, 3.0, 1.0, 2.0, 1.5]"
4,5,"[5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, ..."
...,...,...
9719,193581,[4.0]
9720,193583,[3.5]
9721,193585,[3.5]
9722,193587,[3.5]


In [ ]:
movies_test = movies.copy()
movies_test1 = pd.merge(movies_test,df1,on='movieId')
movies_test1

,movieId,title,genres,rating_list
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[4.0, 4.0, 4.5, 2.5, 4.5, 3.5, 4.0, 3.5, 3.0, ..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.5, 4.0, ..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, ..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[3.0, 3.0, 3.0, 3.0, 1.0, 2.0, 1.5]"
4,5,Father of the Bride Part II (1995),Comedy,"[5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, ..."
...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,[4.0]
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,[3.5]
9721,193585,Flint (2017),Drama,[3.5]
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,[3.5]


In [ ]:
sum_data=[]
dataf ={}
for ind in movies_test1.index:
  r=movies_test1['rating_list'][ind]
  # m_id=movies['movieId'][ind]
  my_ad = [int(1) if x in r else int(0) for x in rating_list]
  my_ad.insert(0,movies_test1['title'][ind])
  sum_data.append(my_ad)
#sum_data
data_t = pd.DataFrame(np.array(sum_data),columns=rating_list_, index=movies_test1['movieId'])
data_t

,title,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
movieId,,,,,,,,,,,
1,Toy Story (1995),1,0,1,1,1,1,1,1,1,1
2,Jumanji (1995),1,1,1,1,1,1,1,1,1,1
3,Grumpier Old Men (1995),1,1,1,1,1,1,1,1,0,1
4,Waiting to Exhale (1995),0,1,1,1,0,1,0,0,0,0
5,Father of the Bride Part II (1995),1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),0,0,0,0,0,0,0,1,0,0
193583,No Game No Life: Zero (2017),0,0,0,0,0,0,1,0,0,0
193585,Flint (2017),0,0,0,0,0,0,1,0,0,0


In [ ]:
data = data_t.copy()
data.drop(['title'], axis=1, inplace=True)
data = data.astype(int)
data

,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
movieId,,,,,,,,,,
1,1,0,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,0,1
4,0,1,1,1,0,1,0,0,0,0
5,1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
193581,0,0,0,0,0,0,0,1,0,0
193583,0,0,0,0,0,0,1,0,0,0
193585,0,0,0,0,0,0,1,0,0,0


In [ ]:
data1= data.copy()
Lawrence_I=data1.loc[[4, 1350,45662,1997,2153,89745,3409,3527,4085]]
sum_= Lawrence_I.sum()
sum_1 = sum_.apply(lambda x: int(0) if x ==0 else int(1))
list(sum_1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
LI=data1.loc[[185,2081,2797]]
LI.values.tolist()

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
df2=pd.DataFrame(np.array(sum_1),index=rating_list,columns=['sum'])
datad = data.dot(df2)

datad['title'] = data_t['title']
datad.reset_index().sort_values(by = ['sum'],ascending = False).head(20)

,movieId,sum,title
156,185,10,"Net, The (1995)"
1544,2081,10,"Little Mermaid, The (1989)"
2102,2797,10,Big (1988)
474,541,10,Blade Runner (1982)
3226,4367,10,Lara Croft: Tomb Raider (2001)
3228,4369,10,"Fast and the Furious, The (2001)"
1525,2058,10,"Negotiator, The (1998)"
7439,81591,10,Black Swan (2010)
3282,4447,10,Legally Blonde (2001)
6868,63082,10,Slumdog Millionaire (2008)
